In [2]:
import pandas as pd
import numpy as np
import tensorflow as tf
import os
import matplotlib.pyplot as plt

from transformers import BertTokenizer, TFAlbertModel
from sklearn.model_selection import train_test_split
from tensorflow.keras.optimizers import Adam
from imblearn.under_sampling import RandomUnderSampler
from tensorflow.keras.layers import Input, Dense

c:\Users\LENOVO\GitHub\capstone_bangkit\venv\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
cwd = os.getcwd()
test = os.path.join(cwd, 'content')
real_df = pd.read_csv(os.path.join(test, 'cnn_content_2023-11-11.csv'))
df = real_df[['berita']]
df

,berita
0,Majelis Kehormatan Mahkamah Konstitusi (MKMK) ...
1,Majelis Kehormatan Mahkamah Konstitusi (MKMK) ...
2,Mahfud lekat dengan inisial MD yang tersemat d...
3,Perebutan tiket atau yang populer dengan istil...
4,Riuh Pemilu 2024 tak cuma seputar capres-cawap...
5,Menteri Kesehatan (Menkes) Budi Gunadi Sadikin...
6,Bakal calon presiden Koalisi Perubahan Anies B...
7,Menteri Pertahanan Prabowo Subianto menyebut a...
8,Sembilan hakim Mahkamah Konstitusi (MK) dikena...
9,Undang-undang Nomor 7 Tahun 2017 tentang Pemil...


In [4]:
test_twt = {
    'berita' : ["lewat di tl aku, dan aku kaget. Aku kira RS ini dibangunnya sama pemerintah kita ternyata pembangunannya dari uang donasi masyarakat kita toh aku nangis banget pas liat videonya juga", "Jog dalam kota kecepatan 80km/jam dan nerobos lampu merah. Dimf bawah ini juga ada cctv kalau ojol berhenti dipinggir anda tabrak, giliran diminta ganti rugi merasa diperas. Playing victim sekali ya wkwkwk", "Peta hubungan diplomatik Israel dengan negara-negara dunia. Hanya ada satu negara dengan perekonomian raksasa ber-GDP di atas $1 triliun yang belum pernah menjalin relasi sama Israel. Yep, Indonesia.", "Pernah penasaran ga sih kenapa Nabi & Rasul itu diutusnya di Arab, Jordan, & Mesir padahal dunia ini luas? Gambar ini sedikit banyak menjelaskan", "Ada perbedaan pada momen kelahiran anak kedua pasangan Atta Halilintar dan Aurel Hermansyah. Pasalnya, semua orangtua atau nenek dari Ameena dan putri keduanya turut hadir. Tentunya hal ini memberikan kebahagiaan bagi Atta dan Aurel. Apalagi mengingat saat momen Ameena lahir, tak seramai kelahiran anak keduanya ini. Saat ditemui di RSIA Bunda, Jakarta Pusat, Senin (13/11) Atta Halilintar merasa momen kelahiran adik Ameena seperti mimpi yang menjadi kenyataan baginya. “Ini udah kayak mimpi jadi nyata buat saya sama Aurel karena semua orangtua bisa hadir. Kemarin (anak pertama) lahiran kan pas Covid ya jadi nggak semua bisa hadir,” ungkap Atta Halilintar. 1. Didampingi Orang-Orang Tersayang Diketahui Dalam foto yang dibagikan di akun media sosial Atta Halilintar, ayah dan ibunya tampak hadir di rumah sakit. Begitu juga dengan orangtua Aurel, Anang Hermansyah dan Ashanty serta Krisdayanti yang tak ingin ketinggalan momen tersebut. Atta pun mengatakan, orangtua dan mertuanya hadir di rumah sakit menanti detik-detik kelahiran anak keduanya. Bahkan mereka juga rela berangkat dini hari ke rumah sakit. 2. Berkumpul Sejak Subuh “Semua mendoakan, semua lengkap semuanya udah ada dari pagi ngumpul di RSIA Bunda mengantar ke rumah sakit dari jam 03.00 WIB terus salat subuh bareng juga,” ujarnya. Ia menilai doa dari orangtuanya dan mertuanya menjadi kekuatan bagi Aurel Hermansyah. “Jadi menurutku doa dari semua jadi menguatkan juga ya. Semua sudah nggak sabar menyambut cucu keduanya,” tukasnya."] 
}

df_test_twt = pd.DataFrame(test_twt)
df_test_twt

df = pd.concat([real_df[['berita']], df_test_twt], ignore_index=True)
df

,berita
0,Majelis Kehormatan Mahkamah Konstitusi (MKMK) ...
1,Majelis Kehormatan Mahkamah Konstitusi (MKMK) ...
2,Mahfud lekat dengan inisial MD yang tersemat d...
3,Perebutan tiket atau yang populer dengan istil...
4,Riuh Pemilu 2024 tak cuma seputar capres-cawap...
5,Menteri Kesehatan (Menkes) Budi Gunadi Sadikin...
6,Bakal calon presiden Koalisi Perubahan Anies B...
7,Menteri Pertahanan Prabowo Subianto menyebut a...
8,Sembilan hakim Mahkamah Konstitusi (MK) dikena...
9,Undang-undang Nomor 7 Tahun 2017 tentang Pemil...


In [5]:
df.drop_duplicates(inplace=True)
df.dropna(inplace = True)

df_test = df

max_len = 70
model_name = 'indobenchmark/indobert-lite-base-p1'
tokenizer = BertTokenizer.from_pretrained(model_name)

X_test = tokenizer(
    text=df['berita'].tolist(),
    add_special_tokens=True,
    max_length=max_len,
    truncation=True,
    padding=True,
    return_tensors='tf',
    return_token_type_ids=False,
    return_attention_mask=True,
    verbose=True
)

X_test

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'AlbertTokenizer'. 
The class this function is called from is 'BertTokenizer'.


{'input_ids': <tf.Tensor: shape=(25, 70), dtype=int32, numpy=
array([[    2,  6810,  9293, ...,   680, 29949,     3],
       [    2,  6810,  9293, ...,  4887,  7343,     3],
       [    2, 20469, 26243, ...,   126,  1328,     3],
       ...,
       [    2,  2577,  1314, ...,     0,     0,     0],
       [    2,   746,  5728, ...,     0,     0,     0],
       [    2,   176,  2217, ...,   102,  6455,     3]])>, 'attention_mask': <tf.Tensor: shape=(25, 70), dtype=int32, numpy=
array([[1, 1, 1, ..., 1, 1, 1],
       [1, 1, 1, ..., 1, 1, 1],
       [1, 1, 1, ..., 1, 1, 1],
       ...,
       [1, 1, 1, ..., 0, 0, 0],
       [1, 1, 1, ..., 0, 0, 0],
       [1, 1, 1, ..., 1, 1, 1]])>}

In [14]:
# single inference

# interpreter = tf.lite.Interpreter(model_path='models/converted_quant_model_litebert.tflite')
# interpreter.allocate_tensors()

# input_details = interpreter.get_input_details()
# output_details = interpreter.get_output_details()

# input_shape = input_details[0]['shape']
# input_data = tf.expand_dims(X_test['input_ids'][0], axis = 0)
# input_intent = tf.expand_dims(X_test['attention_mask'][0], axis = 0)

# interpreter.set_tensor(input_details[0]['index'], input_data)
# interpreter.set_tensor(input_details[1]['index'], input_intent)

# interpreter.invoke()

# output_data = interpreter.get_tensor(output_details[0]['index'])
# print(output_data)

In [18]:
# batch inference

interpreter = tf.lite.Interpreter(model_path='models/converted_quant_model_litebert.tflite')
interpreter.allocate_tensors()

input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()

input_shape = input_details[0]['shape']

output_data = []

for i in range(len(df)):
    input_data = tf.expand_dims(X_test['input_ids'][i], axis = 0)
    input_intent = tf.expand_dims(X_test['attention_mask'][i], axis = 0)
    interpreter.set_tensor(input_details[0]['index'], input_data)
    interpreter.set_tensor(input_details[1]['index'], input_intent)
    interpreter.invoke()
    output_data.append(interpreter.get_tensor(output_details[0]['index']))

print(output_data)

[array([[0.03551087]], dtype=float32), array([[0.04580806]], dtype=float32), array([[0.04104505]], dtype=float32), array([[0.04161768]], dtype=float32), array([[0.03933087]], dtype=float32), array([[0.03777657]], dtype=float32), array([[0.03521077]], dtype=float32), array([[0.14775464]], dtype=float32), array([[0.03545364]], dtype=float32), array([[0.04476575]], dtype=float32), array([[0.0717652]], dtype=float32), array([[0.03409308]], dtype=float32), array([[0.08206773]], dtype=float32), array([[0.04468146]], dtype=float32), array([[0.06191778]], dtype=float32), array([[0.04697829]], dtype=float32), array([[0.05303025]], dtype=float32), array([[0.02611485]], dtype=float32), array([[0.07328288]], dtype=float32), array([[0.04554227]], dtype=float32), array([[0.07112785]], dtype=float32), array([[0.09130548]], dtype=float32), array([[0.07385018]], dtype=float32), array([[0.04648475]], dtype=float32), array([[0.0671972]], dtype=float32)]


In [19]:
output_data

[array([[0.03551087]], dtype=float32),
 array([[0.04580806]], dtype=float32),
 array([[0.04104505]], dtype=float32),
 array([[0.04161768]], dtype=float32),
 array([[0.03933087]], dtype=float32),
 array([[0.03777657]], dtype=float32),
 array([[0.03521077]], dtype=float32),
 array([[0.14775464]], dtype=float32),
 array([[0.03545364]], dtype=float32),
 array([[0.04476575]], dtype=float32),
 array([[0.0717652]], dtype=float32),
 array([[0.03409308]], dtype=float32),
 array([[0.08206773]], dtype=float32),
 array([[0.04468146]], dtype=float32),
 array([[0.06191778]], dtype=float32),
 array([[0.04697829]], dtype=float32),
 array([[0.05303025]], dtype=float32),
 array([[0.02611485]], dtype=float32),
 array([[0.07328288]], dtype=float32),
 array([[0.04554227]], dtype=float32),
 array([[0.07112785]], dtype=float32),
 array([[0.09130548]], dtype=float32),
 array([[0.07385018]], dtype=float32),
 array([[0.04648475]], dtype=float32),
 array([[0.0671972]], dtype=float32)]

In [20]:
model = tf.keras.models.load_model('models/model_2023-11-29', custom_objects={"TFAlbertModel": TFAlbertModel})
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_ids (InputLayer)      [(None, 70)]                 0         []                            
                                                                                                  
 attention_mask (InputLayer  [(None, 70)]                 0         []                            
 )                                                                                                
                                                                                                  
 tf_albert_model (TFAlbertM  TFBaseModelOutputWithPooli   1168358   ['input_ids[0][0]',           
 odel)                       ng(last_hidden_state=(None   4          'attention_mask[0][0]']      
                             , 70, 768),                                                      

In [21]:
optimizer = Adam(
    learning_rate=5e-05, # HF recommendation
    epsilon=1e-08,
    clipnorm=1.0
)

loss = 'binary_crossentropy'
metric = 'accuracy'

model.compile(
    optimizer=optimizer,
    loss=loss,
    metrics=metric
)

In [22]:
predicted = model.predict({'input_ids': X_test['input_ids'], 'attention_mask': X_test['attention_mask']})

1/1 [==============================] - 12s 12s/step


In [23]:
predicted

array([[0.01053031],
       [0.01285332],
       [0.0161338 ],
       [0.02264921],
       [0.00804643],
       [0.02948184],
       [0.00513424],
       [0.08487377],
       [0.04703905],
       [0.04702058],
       [0.02074589],
       [0.402789  ],
       [0.02003481],
       [0.92463464],
       [0.07052504],
       [0.00991829],
       [0.01097579],
       [0.5808339 ],
       [0.00552474],
       [0.01489791],
       [0.9988773 ],
       [0.9988525 ],
       [0.16859588],
       [0.99874526],
       [0.03738075]], dtype=float32)

In [24]:
len(predicted), len(output_data)

(25, 25)

In [28]:
output_data = np.expand_dims([data.squeeze() for data in output_data], axis = 1)
output_data

array([[0.03551087],
       [0.04580806],
       [0.04104505],
       [0.04161768],
       [0.03933087],
       [0.03777657],
       [0.03521077],
       [0.14775464],
       [0.03545364],
       [0.04476575],
       [0.0717652 ],
       [0.03409308],
       [0.08206773],
       [0.04468146],
       [0.06191778],
       [0.04697829],
       [0.05303025],
       [0.02611485],
       [0.07328288],
       [0.04554227],
       [0.07112785],
       [0.09130548],
       [0.07385018],
       [0.04648475],
       [0.0671972 ]], dtype=float32)

In [29]:
predicted - output_data

array([[-0.02498056],
       [-0.03295474],
       [-0.02491125],
       [-0.01896848],
       [-0.03128444],
       [-0.00829474],
       [-0.03007653],
       [-0.06288087],
       [ 0.01158541],
       [ 0.00225483],
       [-0.05101931],
       [ 0.3686959 ],
       [-0.06203292],
       [ 0.87995315],
       [ 0.00860726],
       [-0.03706001],
       [-0.04205446],
       [ 0.5547191 ],
       [-0.06775813],
       [-0.03064437],
       [ 0.92774945],
       [ 0.907547  ],
       [ 0.0947457 ],
       [ 0.9522605 ],
       [-0.02981644]], dtype=float32)